In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msgn
import time
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import *

from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold, train_test_split, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier

import sklearn
from tqdm import tqdm
from sklearn.base import ClassifierMixin
from sklearn.utils.testing import all_estimators

In [2]:
df = pd.read_pickle('df_v1_01.pickle')

In [3]:
df.isna().sum()

order_date                                  0
user_id                                     0
is_elite_user                               0
supplier_id                                 0
order_line_item_id                          0
                                       ...   
WEB_qa-supplier                       7642121
rate_reviews-supplier                  450920
review_like_count_reviews-supplier     450920
pos_comment_reviews-supplier           450920
neg_comment_reviews-supplier           450920
Length: 78, dtype: int64

In [4]:
df.head(5)

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_variant_id,original_price,discounted_price,ship_cost,...,pos_answers_qa-supplier,neg_answers_qa-supplier,ANDROID_qa-supplier,IOS_qa-supplier,MOBILE_WEB_qa-supplier,WEB_qa-supplier,rate_reviews-supplier,review_like_count_reviews-supplier,pos_comment_reviews-supplier,neg_comment_reviews-supplier
0,2021-05-19 01:27:28.768000+00:00,2141,0,216503,955598029,625610651,133066437,104.900002,69.900002,11.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-19 01:27:28.768000+00:00,2141,0,1188,955598030,625610651,77047570,79.989998,79.989998,8.88,...,7.0,0.0,2.0,0.0,0.0,0.0,4.736842,24.0,2.026316,1.197368
2,2021-06-02 00:51:35.862000+00:00,2141,0,200788,988146564,643936439,153411316,110.300003,55.150002,8.88,...,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,0.0,1.000000,0.000000
3,2021-06-02 00:51:35.862000+00:00,2141,0,107296,988146563,643936439,172744639,59.900002,49.900002,8.88,...,3.0,6.0,5.0,7.0,0.0,0.0,4.450000,43.0,1.933333,0.933333
4,2021-06-09 23:31:37.963000+00:00,2141,0,968,1007223734,654756664,162306389,119.989998,48.000000,9.40,...,NaN,NaN,NaN,NaN,NaN,NaN,4.384615,1.0,1.807692,0.923077


In [5]:
label = ['is_returned']

categorical_str = [
'is_saved_card_trx',
'gender',
'zodiac',
'ANDROID',
'ANDROID_qa-supplier',
'IOS',
'IOS_qa-supplier',
'MOBILE_WEB',
'MOBILE_WEB_qa-supplier',
'WEB',
'WEB_qa-supplier',
'rate',
'rate_reviews-supplier',
] 

categorical_int = [
'gender_id',
'category_id',
'color_id',
'order_weekday',
'order_week',
'brand_id',
'order_day',
'hour_interval',
]

numerical = [
'n_unq_variants',
'original_price',
'discounted_price',
'ship_cost',
'coupon_discount',
'age',
'diff_order_memdate',
'total_claim',
'promotion_award_value',
'discount_ratio',
'paid_amount',
'beden_comment',
'beden_comment_qa-supplier',
'beden_question',
'beden_question_qa-supplier',
'kalite_comment',
'kalite_comment_qa-supplier',
'kalite_question',
'kalite_question_qa-supplier',
'neg_answers',
'neg_answers_qa-supplier',
'neg_comment',
'neg_comment_reviews-supplier',
'pos_answers',
'pos_answers_qa-supplier',
'pos_comment',
'pos_comment_reviews-supplier',
'review_like_count',
'review_like_count_reviews-supplier'
]

numerical2 = [
'returnRate',
'returnRate_def',
'unresolvedclaim_percentage',
'unresolved_percentage',
]
passthrough = [
'is_elite_user',
'is_wallet_trx',
'is_bday_close',

]


drops = [
'color_name',
'supplier_color_name',
'attribute_value',
'attributet_name',
'gender_name',
'category_name',
'brand_name',
'product_name',
'coupon_id',
'promotion_name',
'order_date',
'birth_date',
'membership_date', 
'order_line_item_id',
'order_parent_id',
'product_content_id',
'product_id',
'product_variant_id',
'supplier_id',
'user_id']

In [6]:
set(df.columns).difference(set(categorical_str+categorical_int+numerical+numerical2+passthrough+label+drops))

set()

In [7]:
set(categorical_str+categorical_int+numerical+numerical2+passthrough+label+drops).difference(set(df.columns))

set()

In [ ]:
null_df = df[df['is_returned'].isna()]
notna_df = df[df['is_returned'].notna()]

In [ ]:
del df

In [ ]:
import gc
gc.collect()

In [ ]:
print(null_df['is_returned'].isna().sum()), print(notna_df['is_returned'].isna().sum())

In [ ]:
null_df.isna().sum()

In [ ]:
len(null_df['product_content_id'].unique())

In [ ]:
null_df.drop_duplicates(subset=['order_parent_id', 'product_content_id'], inplace=True)

In [ ]:
null_df.shape

In [ ]:
merged_df = notna_df[categorical_str+categorical_int+numerical+numerical2+passthrough+label]

matrix = merged_df.corr().abs()

# Create a mask
mask = np.triu(np.ones_like(matrix, dtype=bool))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)
plt.figure(figsize=(16, 12))
sns.heatmap(matrix[matrix>0.5], mask=mask, center=0, annot=True, fmt='.2f', square=True, cmap='crest')

plt.show();

veriyi alırken

1. random
2. time based

In [ ]:
pos_df = merged_df[merged_df['is_returned']==1].sample(100000, random_state=1)
neg_df = merged_df[merged_df['is_returned']==0].sample(100000, random_state=1)

model_df = pd.concat([pos_df, neg_df], axis=0)
model_df.sample(frac=1)

y = model_df['is_returned']
X = model_df[categorical_str+categorical_int+numerical+numerical2+passthrough]

In [ ]:
def print_metrics(y_pred, y_test, model):
    print("Model results: ", type(model).__name__)
    print(classification_report(y_test, y_pred))
    print("Confusion matrix:")
    cm = confusion_matrix(y_test,y_pred, labels= model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels= model.classes_)
    disp.plot()

In [ ]:
def create_pipeline_rf():
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    numeric_transformer2 = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))])
    
    categorical_str_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    
    categorical_int_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=-99)),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    

    preprocessor = ColumnTransformer(
                                    transformers=[
                                                  ('num1', numeric_transformer, numerical),
                                                  ('num2', numeric_transformer2, numerical2),
                                                  ('cat_str1', categorical_str_transformer, categorical_str),
                                                  ('cat_int1', categorical_int_transformer, categorical_int),
                                                  ], 
                                    remainder='passthrough')

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('f_selector', SelectKBest(f_classif, k='all')),
                          ('classifier', RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0))])
    
    return pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=1)

pipe1 = create_pipeline_rf()

pipe1.fit(X_train, y_train)
preds = pipe1.predict(X_test)

In [ ]:
print_metrics(preds, y_test, pipe1)

### feature importance

In [ ]:
%matplotlib inline
#do code to support model
#"data" is the X dataframe and model is the SKlearn object

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X.columns, pipe1['classifier'].feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'importance'})
importances.sort_values(by='importance').plot(kind='barh',figsize=(8,8))

## auto ml

In [ ]:
def evaluate_model_with_CV(X, y, model):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    scores = cross_validate(pipeline, X, y, scoring=['accuracy', 'f1_weighted'], n_jobs=-1, 
                        return_estimator=True, return_train_score=True, cv=cv)
    # ['fit_time', 'score_time', 'estimator', 'test_accuracy', 'train_accuracy', 'test_f1_weighted', 'train_f1_weighted']
    return scores

In [ ]:
def print_metrics(y_pred, y_test, model):
    print("Model results: ", type(model).__name__)
    print(classification_report(y_test, y_pred))
    print("Confusion matrix:")
    cm = confusion_matrix(y_test,y_pred, labels= model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels= model.classes_)
    disp.plot()

In [ ]:
def create_pipeline_without_clf(clf_name, clf_object):
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    numeric_transformer2 = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))])
    
    categorical_str_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    
    categorical_int_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=-99)),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    

    preprocessor = ColumnTransformer(
                                    transformers=[
                                                  ('num1', numeric_transformer, numerical),
                                                  ('num2', numeric_transformer2, numerical2),
                                                  ('cat_str1', categorical_str_transformer, categorical_str),
                                                  ('cat_int1', categorical_int_transformer, categorical_int),
                                                  ], 
                                    remainder='passthrough')

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('f_selector', SelectKBest(f_classif, k='all')),
                          (clf_name, clf_object)], verbose=False)
    
    return pipeline

In [ ]:
CLASSIFIERS = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('KNeighborsClassifier', sklearn.neighbors._classification.KNeighborsClassifier),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('XGBClassifier', XGBClassifier),
 ('LGBMClassifier', LGBMClassifier)]

In [ ]:
random_state = 10
Accuracy_mean = []
Accuracy_std = []
F1_mean = []
F1_std = []
names = []
TIME = []
predictions = {}
models = {}

for name, model in tqdm(CLASSIFIERS):
    start = time.time()
    try:
        if "random_state" in model().get_params().keys():
            if "class_weight" in model().get_params().keys():
                pipeline = create_pipeline_without_clf("classifier", model(random_state=random_state, class_weight='balanced'))
            else:
                pipeline = create_pipeline_without_clf("classifier", model(random_state=random_state))
        else:
            if "class_weight" in model().get_params().keys():
                pipeline = create_pipeline_without_clf("classifier", model(class_weight='balanced'))
            else:
                pipeline = create_pipeline_without_clf("classifier", model())
                
                                   
        results = evaluate_model_with_CV(X, y, model)
                        
        names.append(name)
        Accuracy_mean.append(results['test_accuracy'].mean())
        Accuracy_std.append(results['test_accuracy'].std())
        F1_mean.append(results['test_f1_weighted'].mean())
        F1_std.append(results['test_f1_weighted'].std())
        TIME.append(results['fit_time'].mean())
    
    except Exception as exception:
        print(name + " model failed to execute")
        print(exception)
                                   
scores = pd.DataFrame(
    {   "Model": names,
        "Accuracy Mean": Accuracy_mean,
        "Accuracy Std": Accuracy_std,
        "F1-Score Mean": F1_mean,
        "F1-Score Std": F1_std,
        "Time Taken": TIME, })

In [ ]:
scores.sort_values(by='F1-Score Mean', ascending=False)

In [ ]:
pos_df = merged_df[merged_df['is_returned']==1].sample(50000, random_state=1)
neg_df = merged_df[merged_df['is_returned']==0].sample(50000, random_state=1)

model_df = pd.concat([pos_df, neg_df], axis=0)
model_df.sample(frac=1)

y = model_df['is_returned']
X = model_df[categorical_str+categorical_int+numerical+numerical2+passthrough]


In [ ]:
random_state = 10
Accuracy_mean = []
Accuracy_std = []
F1_mean = []
F1_std = []
names = []
TIME = []
predictions = {}
models = {}

for name, model in tqdm(CLASSIFIERS):
    start = time.time()
    try:
        if "random_state" in model().get_params().keys():
            if "class_weight" in model().get_params().keys():
                pipeline = create_pipeline_without_clf("classifier", model(random_state=random_state, class_weight='balanced'))
            else:
                pipeline = create_pipeline_without_clf("classifier", model(random_state=random_state))
        else:
            if "class_weight" in model().get_params().keys():
                pipeline = create_pipeline_without_clf("classifier", model(class_weight='balanced'))
            else:
                pipeline = create_pipeline_without_clf("classifier", model())
                
                                   
        results = evaluate_model_with_CV(X, y, model)
                        
        names.append(name)
        Accuracy_mean.append(results['test_accuracy'].mean())
        Accuracy_std.append(results['test_accuracy'].std())
        F1_mean.append(results['test_f1_weighted'].mean())
        F1_std.append(results['test_f1_weighted'].std())
        TIME.append(results['fit_time'].mean())
    
    except Exception as exception:
        print(name + " model failed to execute")
        print(exception)
                                   
scores2 = pd.DataFrame(
    {   "Model": names,
        "Accuracy Mean": Accuracy_mean,
        "Accuracy Std": Accuracy_std,
        "F1-Score Mean": F1_mean,
        "F1-Score Std": F1_std,
        "Time Taken": TIME, })

In [ ]:
scores2.sort_values(by='F1-Score Mean', ascending=False)

In [ ]:
XGBClassifier
LGBMClassifier

In [ ]:
different encodings
    category_name'i encoding? one-hot
    
    encodings:
    
    Frequency Encoding

    Replace the values with its frequency
    But be careful, some values may have same frequency
    data[‘country’].value_counts()

    Target Encoding (Mean encoding)

    Each of the categories is the variable is replaced with the mean target value for that category
    for each catergory in a column: sum of target / count of target 

## validation

In [ ]:
data_path = r'C:\Users\IsmailKaraman\workspace\data\trendyol\\'
test_raw = pd.read_csv(data_path+'test_data.csv')
test_raw.head(2)

In [ ]:
test = test_raw.copy()
test[['user_id', 'product_content_id', 'order_parent_id']] = test['id'].str.split('|',expand=True)
test = test[['user_id', 'product_content_id', 'order_parent_id','expected']]

In [ ]:
test.head(3)

In [ ]:
test.info()

In [ ]:
test['user_id'] = test['user_id'].astype('int64') 
test['order_parent_id'] = test['order_parent_id'].astype('int64')
test['product_content_id'] = test['product_content_id'].astype('int64')

In [ ]:
null_df.shape

In [ ]:
test_df = pd.merge(test, null_df, on=['user_id', 'product_content_id', 'order_parent_id'], how='left')

In [ ]:
test_df.shape

In [ ]:
test_df.isna().sum()

In [ ]:
merged_df.columns

In [ ]:
test_X = test_df[categorical_str+categorical_int+numerical+numerical2+passthrough]

In [ ]:
test_X.isna().sum()

In [ ]:
expected = pipe1.predict(test_df[categorical_str+categorical_int+numerical+numerical2+passthrough])

In [ ]:
test_raw['expected'] = expected

In [ ]:
test_raw['expected'] = test_raw['expected'].astype('int32')

In [ ]:
test_raw.to_csv('output.csv', index=False)